## neo4j-contrib:neo4j-spark-connector versions
https://mvnrepository.com/artifact/neo4j-contrib/neo4j-spark-connector

In [1]:
%%init_spark
# Configure Spark to use a local master
# launcher.master = "local[*]"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.4.0-M6"

# needs java 1.8
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.2.1-M5"

# saveGraph mixes up id and number (for scala 2.11)
launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M4"

# saveGraph mixes up id and number (for scala 2.10)
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.1.0-M3"

# doesnt have neo.saveGraph()
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M2" 

# doesnt do neo = Neo4j(sc)
# launcher.packages = "neo4j-contrib:neo4j-spark-connector:2.0.0-M1"  

In [2]:
println(sc.master)
// val rdd = sc.parallelize(0 to 999)
// rdd.takeSample(false, 5)

Intitializing Scala interpreter ...

Spark Web UI available at http://hadoop-master:8088/proxy/application_1552053568571_0002
SparkContext available as 'sc' (version = 2.0.0, master = yarn, app id = application_1552053568571_0002)
SparkSession available as 'spark'


yarn


In [3]:
import org.neo4j.spark._

val neo = Neo4j(sc)

import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._

import org.neo4j.spark._
neo: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@71f00b67
import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._


In [4]:
// load graph via Cypher query
// val graphQuery = "MATCH (n:Person)-[r:KNOWS]->(m:Person) RETURN id(n) as source, id(m) as target, type(r) as value SKIP {_skip} LIMIT {_limit}"
// val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(7).batch(200).loadGraph
val graphQuery = "MATCH (n:touchedBySpark)-[r:touchedBySpark_rel]->(m:touchedBySpark) RETURN n.my_id as source, m.my_id as target, type(r) as value"
val graph: Graph[Long, String] = neo.rels(graphQuery).partitions(2).batch(15).loadGraph
println(graph.vertices.count)
println(graph.edges.count)

27
82


graphQuery: String = MATCH (n:touchedBySpark)-[r:touchedBySpark_rel]->(m:touchedBySpark) RETURN n.my_id as source, m.my_id as target, type(r) as value
graph: org.apache.spark.graphx.Graph[Long,String] = org.apache.spark.graphx.impl.GraphImpl@6103ec11


In [5]:
// val graph2_pagerank = PageRank.run(graph2,500) //java.lang.OutOfMemoryError: Java heap space
val graph_pagerank = PageRank.run(graph,100)

graph_pagerank: org.apache.spark.graphx.Graph[Double,Double] = org.apache.spark.graphx.impl.GraphImpl@5b52cafb


In [6]:
graph.edges.collect()

res2: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(0,1,touchedBySpark_rel), Edge(0,2,touchedBySpark_rel), Edge(0,16,touchedBySpark_rel), Edge(3,4,touchedBySpark_rel), Edge(3,6,touchedBySpark_rel), Edge(3,16,touchedBySpark_rel), Edge(4,1,touchedBySpark_rel), Edge(5,4,touchedBySpark_rel), Edge(5,11,touchedBySpark_rel), Edge(6,2,touchedBySpark_rel), Edge(7,6,touchedBySpark_rel), Edge(7,11,touchedBySpark_rel), Edge(8,4,touchedBySpark_rel), Edge(8,11,touchedBySpark_rel), Edge(9,10,touchedBySpark_rel), Edge(10,5,touchedBySpark_rel), Edge(11,12,touchedBySpark_rel), Edge(12,13,touchedBySpark_rel), Edge(13,14,touchedBySpark_rel), Edge(14,15,touchedBySpark_rel), Edge(16,15,touchedBySpark_rel), Edge(17,19,touchedBySpark_rel), Edge(17,20,touchedBySpark_rel), Edge(18,20,touchedBySpark_re...

In [7]:
graph.vertices.collect()

res3: Array[(org.apache.spark.graphx.VertexId, Long)] = Array((4,1), (16,15), (22,25), (14,15), (0,2), (24,26), (6,2), (8,11), (12,13), (18,20), (20,22), (26,18), (10,5), (2,0), (13,14), (19,21), (21,23), (15,0), (25,18), (11,12), (23,25), (1,0), (17,20), (3,6), (7,11), (9,10), (5,4))


In [8]:
graph_pagerank.edges.collect()

res4: Array[org.apache.spark.graphx.Edge[Double]] = Array(Edge(0,1,0.16666666666666666), Edge(0,2,0.16666666666666666), Edge(0,16,0.16666666666666666), Edge(3,4,0.16666666666666666), Edge(3,6,0.16666666666666666), Edge(3,16,0.16666666666666666), Edge(4,1,0.5), Edge(5,4,0.25), Edge(5,11,0.25), Edge(6,2,0.5), Edge(7,6,0.25), Edge(7,11,0.25), Edge(8,4,0.25), Edge(8,11,0.25), Edge(9,10,0.5), Edge(10,5,0.5), Edge(11,12,0.5), Edge(12,13,0.5), Edge(13,14,0.5), Edge(14,15,0.5), Edge(16,15,0.5), Edge(17,19,0.25), Edge(17,20,0.25), Edge(18,20,0.25), Edge(18,21,0.25), Edge(19,21,0.25), Edge(19,22,0.25), Edge(20,22,0.25), Edge(20,23,0.25), Edge(21,23,0.25), Edge(21,24,0.25), Edge(22,24,0.25), Edge(22,25,0.25), Edge(23,25,0.25), Edge(23,26,0.25), Edge(24,17,0.25), Edge(24,26,0.25), Edge(25,17,0.25),...

In [9]:
graph_pagerank.vertices.collect()

res5: Array[(org.apache.spark.graphx.VertexId, Double)] = Array((4,0.420246875), (16,0.235), (22,0.9999999256447741), (14,0.657009268359375), (0,0.15), (24,0.9999999256447741), (6,0.25625), (8,0.15), (12,0.52527234375), (18,0.9999999256447741), (20,0.9999999256447741), (26,0.9999999256447741), (10,0.27749999999999997), (2,0.41031249999999997), (13,0.5964814921875), (19,0.9999999256447741), (21,0.9999999256447741), (15,0.9082078781054688), (25,0.9999999256447741), (11,0.441496875), (23,0.9999999256447741), (1,0.54970984375), (17,0.9999999256447741), (3,0.15), (7,0.15), (9,0.15), (5,0.38587499999999997))


In [10]:
--

<console>: 35: error: not found: value --

In [11]:
val mypattern_cmon = neo.pattern(("touchedBySpark","my_id"),("touchedBySpark_rel",null),("touchedBySpark","my_id"))

mypattern_cmon: org.neo4j.spark.Neo4j = org.neo4j.spark.Neo4j@71f00b67


In [12]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph_pagerank, "rankcmon", mypattern_cmon.pattern, merge = false)

res7: Long = 246


In [ ]:
// uses pattern from above to save the data, merge parameter is false by default, only update existing nodes
neo.saveGraph(graph_pagerank, "rankcmon")

In [16]:
import scala.math._

import scala.math._


In [15]:
PageRank.run?